# ***CRAWLING DATA TWITTER***



In [ ]:
#@title Twitter Auth Token

twitter_auth_token = 'put twt auth token here' # change this auth token

In [ ]:
# Import required Python package
!pip install pandas

# Install Node.js (because tweet-harvest built using Node.js)
!sudo apt-get update
!sudo apt-get install -y ca-certificates curl gnupg
!sudo mkdir -p /etc/apt/keyrings
!curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg

!NODE_MAJOR=20 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list

!sudo apt-get update
!sudo apt-get install nodejs -y

!node -v

In [ ]:
# Crawl Data

filename = 'iisma.csv'
search_keyword = 'iisma since:2023-04-01 until:2024-04-01 lang:id'
limit = 300

!npx -y tweet-harvest@2.6.1 -o "{filename}" -s "{search_keyword}" --tab "LATEST" -l {limit} --token {twitter_auth_token}

In [ ]:
import pandas as pd

# Specify the path to your CSV file
file_path = f"tweets-data/{filename}"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path, delimiter=",")

# Display the DataFrame
display(df)

In [ ]:
# Cek jumlah data yang didapatkan

num_tweets = len(df)
print(f"Jumlah tweet dalam dataframe adalah {num_tweets}.")

# ***PRE-PROCESSING DATA***

In [ ]:
import pandas as pd

# Specify the path to your CSV file
file_path = f"tweets-data/{filename}"

# Read the CSV file into a pandas DataFrame
data = pd.read_csv(file_path, delimiter=",")

# Display the DataFrame
display(data)
#

In [ ]:
data.info()

In [ ]:
data.drop_duplicates(subset='full_text', keep = 'first', inplace=True)
data.info()

In [ ]:
df = pd.DataFrame(data['full_text'])
df.head(319)

Cleaning

In [ ]:
import re
import string
import nltk

def remove_URL(tweet):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', tweet)

def remove_html(tweet):
    html = re.compile(r'<.*?>')
    return html.sub(r'', tweet)

def remove_emoji(tweet):
    emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"
      u"\U0001F300-\U0001F5FF"
      u"\U0001F680-\U0001F6FF"
      u"\U0001F1E0-\U0001F1FF"
      "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', tweet)

def remove_angka(tweet):
    tweet = re.sub(r'\d+', '', tweet)
    return tweet

def remove_symbols(tweet):
    tweet = re.sub(r'[^a-zA-Z0-9\s]', '', tweet)
    return tweet

df['cleaning'] = df['full_text'].apply(lambda x: remove_URL(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_html(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_emoji(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_symbols(x))
df['cleaning'] = df['cleaning'].apply(lambda x: remove_angka(x))

df.head(319)

CASE FOLDING

In [ ]:
def case_folding(text):
    if isinstance(text, str):
        lowercase_text = text.lower()
        return lowercase_text
    else:
        return text

df['case_folding'] = df['cleaning'].apply(case_folding)
df.head(319)

TOKENIZATION

In [ ]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case_folding'].apply(tokenize)
df.head(319)

FILTERING/STOPWORD REMOVAL

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('indonesian')

In [ ]:
def remove_stopwords(text):
    return [word for word in text if word not in stop]

df['filtering/stopword removal'] = df['tokenize'].apply(lambda x: remove_stopwords(x))
df.head(319)

STEAMING DATA

In [ ]:
!pip install Sastrawi

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem_text(text):
    return [stemmer.stem(word) for word in text]

df['steaming_data'] = df['filtering/stopword removal'].apply(lambda x: ' '.join(stem_text(x)))
df.head(319)

EXPORT PRE-PROCESSED DATA

In [146]:
df.to_csv('preproc.csv', encoding='utf8', index=False)

# ***LABELLING DATA***

In [ ]:
import pandas as pd
import numpy as np

def load_data():
    data = pd.read_csv('preproc.csv')
    return data

data = load_data()
data.head(319)

In [ ]:
data.info()

In [ ]:
df = pd.DataFrame(data['steaming_data'])
df.head(319)

In [ ]:
!pip install nltk
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd

nltk.download('punkt')
nltk.download('vader_lexicon')

In [ ]:
data = SentimentIntensityAnalyzer()
labels = []
scores = []

for text in df['steaming_data']:
    sentiment_scores = data.polarity_scores(text)
    compound_score = sentiment_scores['compound']

    scores.append(compound_score)

    if compound_score > 0:
        labels.append('Positif')
    elif compound_score < 0:
        labels.append('Negatif')
    else:
        labels.append('Netral')

df['sentiment'] = labels
df['sentiment_score'] = scores

data = ['steaming_data', 'sentiment_score', 'sentiment']
data = df[data]

data.head(319)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sentiment_count = data['sentiment'].value_counts()

sns.set_style('whitegrid')

fig, ax = plt.subplots(figsize=(10, 6))
ax = sns.barplot(x=sentiment_count.index, y=sentiment_count.values, palette='pastel')
plt.title('Jumlah Analisis Sentimen', fontsize=14, pad=20)
plt.xlabel('Class Sentiment', fontsize=12)
plt.ylabel('Jumlah Tweet', fontsize=12)

for i, count in enumerate(sentiment_count.values):
    ax.text(i, count + 0.10, str(count), ha='center', va='bottom')

plt.show()

In [153]:
data.to_csv('labelNLTK.csv', encoding='utf8', index=False)